In [1]:
# Dependencies
import json
import requests
from config import client_id, client_secret
import pandas as pd

In [2]:
# Accessing the Json file downloaded from NYU Spatial Data Repository
with open("nyu-2451-34572-geojson.json") as json_data:
    newyork_data = json.load(json_data)
nyc_ndata = newyork_data["features"]

In [3]:
# Creating lists to hold data
borough = []
neigborhood = []
lat = []
lon = []

# Looping thru data and appending to lists
for data in nyc_ndata:
    borough.append(data['properties']['borough'])
    neigborhood.append(data['properties']['name'])
    lat.append(data['geometry']['coordinates'][1])
    lon.append(data['geometry']['coordinates'][0])

# Creating Dataframe to hold all the data
nyc_df = pd.DataFrame({
    "Borough":borough,
    "Neigborhood":neigborhood,
    "Latitude":lat,
    "Longitude":lon
})

In [4]:
nyc_df.head()

,Borough,Neigborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
def createURL(endpoint, client_id, client_secret, version, lat, lon, radius, categoryid, limit):
    url="{}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryid={}&limit={}".format(
    endpoint,
    client_id,
    client_secret,
    v,
    lat,
    lon,
    radius,
    categoryid,
    limit)
    return url

In [40]:
hood_name = nyc_df["Neigborhood"][:3]
lat = nyc_df["Latitude"][:3]
lon = nyc_df["Longitude"][:3]

In [19]:
venues = pd.DataFrame({
    "Venue ID":[],
    "Venue name":[],
    "Category":[],
    "Latitude":[],
    "Longitude":[],
    "State": [],
    "City": [],
    "Neigborhood":[],
    "Likes": [],
    "Checkins": []
})

In [20]:
venues

,Venue ID,Venue name,Category,Latitude,Longitude,State,City,Neigborhood,Likes,Checkins


In [41]:
# Starting URL for Weather Map API Call
endpoint="https://api.foursquare.com/v2/venues/search?"
v=20190607
radius=1000
categoryid = "4bf58dd8d48988d17f941735"
limit=500

total_records = nyc_df["Borough"].count()

# Will keep the count of records
record = 1
index=0

# Log file print statement
print("Beginning Data Retrieval")
print("-------------------------------")


# Loop through the cities in the city list
for hood_name, lat, lon in zip(hood_name, lat, lon):
    
    url=createURL(endpoint, client_id, client_secret, v, lat, lon, radius, categoryid, limit)
    response = requests.get(url).json()["response"]["venues"]
    
    for item in response:
            # Append the data to lists
        try:
            venues.loc[index,"Venue ID"] = item["id"]
            venues.loc[index,"Venue name"] = item["name"]
            venues.loc[index,"Latitude"] = item["location"]["lat"]
            venues.loc[index,"Longitude"] = item["location"]["lng"]
            venues.loc[index,"State"] = item["location"]["state"]
            venues.loc[index,"Neigborhood"] = hood_name
            #venues.loc[index,"City"] = item["location"]['city']
            #venues.loc[index,"Category"] = item["categories"][0]["name"]
        except:
            print("NA")

    print(f"Processing neigborhood {record} out of {total_records} | {hood_name}")
    print(url)

    # Increase counter by one
    record +=1
    index+=1
    
print("Data Retrieval Complete")

Beginning Data Retrieval
-------------------------------
Processing neigborhood 1 out of 306 | Wakefield
https://api.foursquare.com/v2/venues/search?&client_id=ZZOWEUFEZ2LJEQLVHJ3LSFP40KSRDO4INKP5YHA244055N2L&client_secret=FGIZTT1ZG4IZU4NZBMXS1ZTHSBFDKXQJS4EQG0Z2K2QR5OXH&v=20190607&ll=40.89470517661,-73.84720052054902&radius=1000&categoryid=4bf58dd8d48988d17f941735&limit=500
Processing neigborhood 2 out of 306 | Co-op City
https://api.foursquare.com/v2/venues/search?&client_id=ZZOWEUFEZ2LJEQLVHJ3LSFP40KSRDO4INKP5YHA244055N2L&client_secret=FGIZTT1ZG4IZU4NZBMXS1ZTHSBFDKXQJS4EQG0Z2K2QR5OXH&v=20190607&ll=40.87429419303012,-73.82993910812398&radius=1000&categoryid=4bf58dd8d48988d17f941735&limit=500
Processing neigborhood 3 out of 306 | Eastchester
https://api.foursquare.com/v2/venues/search?&client_id=ZZOWEUFEZ2LJEQLVHJ3LSFP40KSRDO4INKP5YHA244055N2L&client_secret=FGIZTT1ZG4IZU4NZBMXS1ZTHSBFDKXQJS4EQG0Z2K2QR5OXH&v=20190607&ll=40.887555677350775,-73.82780644716412&radius=1000&categoryid=4bf58

In [42]:
venues.head()

,Venue ID,Venue name,Category,Latitude,Longitude,State,City,Neigborhood,Likes,Checkins
0,50ac2001e4b0677d4e55892d,Rig's Meat Market,Steakhouse,40.890519,-73.853206,NY,Bronx,Wakefield,NaN,NaN
1,4d8f6fc65091a1cdaef3a501,Co-Op City Bldg 20,Residential Building (Apartment / Condo),40.871358,-73.826402,NY,Bronx,Co-op City,NaN,NaN
2,5283cbba11d26f7fe2ca1c28,Herbalife,Church,40.885262,-73.825374,NY,Bronx,Eastchester,NaN,NaN


In [ ]:
url = "https://api.foursquare.com/v2/venues/search?&client_id=ZZOWEUFEZ2LJEQLVHJ3LSFP40KSRDO4INKP5YHA244055N2L&client_secret=FGIZTT1ZG4IZU4NZBMXS1ZTHSBFDKXQJS4EQG0Z2K2QR5OXH&v=20190607&ll=40.887555677350775,-73.82780644716412&radius=1000&categoryid=4bf58dd8d48988d17f941735&limit=50"
response = requests.get(url).json()["response"]["venues"]
category = []
for item in response:
    try:
        category.append(item["categories"][0]["name"])
    except:
        category.append("na")
        
    

In [ ]:
venues = pd.DataFrame({
    "Venue ID":venue_id,
    "Venue name":name,
    "Category":category,
    "Latitude":latitude,
    "Longitude":longitude,
    "State": state,
    "City": city,
    "Neigborhood":neigborhood
    "Likes": likes,
    "Checkins": checkins
})